In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
def card_info(url):
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    #Headers for the table
    card_info_headers = soup.find_all(attrs = {'class':'cardtablerowheader'})
    headers = []
    for header in card_info_headers:
        headers.append(header.text)
    headers = headers[:-2]
    
    #Information for the table
    card_info_table = soup.find_all(attrs = {'class':'cardtablerowdata'})
    info = []
    for table_entry in card_info_table:
        info.append(table_entry.text)
    info = info[:-2]
    info_2 = []
    for clown in info:
        if '\n' in clown:
            clown_2 = clown.replace('\n','')
            info_2.append(clown_2)
        else:
            info_2.append(clown)
    #Removing the names in different languages
    while not ( info_2[1] == 'Spell ' or info_2[1] == 'Monster ' or info_2[1] == ('Trap ')):
        info_2.remove(info_2[1])
        headers.remove(headers[1])
    else:
        info_2 = info_2
    if ('Traditional' in info_2[-1]) == True:
        if ('TCG Advanced') in info_2[-1]:
            headers.remove(headers[-2])
            info_2.remove(info_2[-2])
        else:
            info_2.remove(info_2[-1])
            info_2.remove(info_2[-2])
            headers.remove(headers[-1])
            headers.remove(headers[-2])
    #Adding the effect
    Jeff = soup.find(attrs = {'class':'collapsible expanded navbox-inner'})
    Jeff = Jeff.text[12:-2]
    info_2.append(Jeff)
    headers.append('Card Text')
    info_dict = {headers[i] : info_2[i] for i in range(0, len(headers))}
    return info_dict

In [2]:
class TCG_set:
    
    def __init__(self, url):
        self.url = url
        result = requests.get(url)
        src = result.content
        soup = BeautifulSoup(src, 'html.parser')
        #Getting the name of the set
        headers_soup = soup.find_all("h2")
        self.set_name = headers_soup[0].text
        #Getting initial set info
        Destiny_1 = soup.find_all('section')
        section_headers = []
        section_headers_txt = []
        for Aster in Destiny_1:
            a = Aster.find_all('h2')
            section_headers.append(a) 
        for header in section_headers:
            if header != []:
                for head in header:
                    section_headers_txt.append(head.text)
            else:
                    section_headers_txt.append('lol')
        #Index of Set Info and Release Date
        ind_1 = section_headers_txt.index('Set Information')
        #Adding dict keys/values to an empty list here
        sub_headers = []
        more_set_info = []
        #Adding set info to the dictionary
        Destiny_2 = Destiny_1[ind_1].find_all(attrs = {'class' : "pi-data-value pi-font"})
        Destiny_4 = Destiny_1[ind_1].find_all(attrs = {'class' : "pi-data-label pi-secondary-font"})
        for Aster in Destiny_2:
            more_set_info.append(Aster.text)
        for Aster in Destiny_4:
            sub_headers.append(Aster.text)
        if 'Release dates' in section_headers_txt:
            ind_3 = section_headers_txt.index('Release dates')
            #Adding release dates to the dictionary
            Destiny_3 = Destiny_1[ind_3].find_all(attrs = {'class' : "pi-data-value pi-font"})
            Destiny_5 = Destiny_1[ind_3].find_all(attrs = {'class' : "pi-data-label pi-secondary-font"})
            for Aster in Destiny_3:
                more_set_info.append(Aster.text)
            for Aster in Destiny_5:
                sub_headers.append(Aster.text + ' Release Date')
        #Making the Dictionary
        sub_head_dict = {sub_headers[i] : more_set_info[i] for i in range(0, len(sub_headers))}
        if 'Prefix(es)' in list(sub_head_dict.keys()):
            set_code = sub_head_dict['Prefix(es)']
            if '-ENSE' in set_code:
                a = set_code.find('-')
                sub_head_dict['Prefix(es)']  = set_code[:a+5]
            else:    
                a = set_code.find('-')
                sub_head_dict['Prefix(es)'] = set_code[:a]
        if ' \(en' in sub_head_dict['Prefix(es)']:
            b = sub_head_dict['Prefix(es)'].find(' ')
            sub_head_dict['Prefix(es)'] = sub_head_dict['Prefix(es)'][:b]
        sub_head_dict['Name'] = headers_soup[0].text
        sub_head_dict['Wiki link'] = url
        self.set_info = sub_head_dict
        #Contents of the set
        Destiny = soup.find_all("table")
        #Get the headers for each table so that we can find each table that we actually want
        tables = []
        for Aster in Destiny:
            bruh = []
            table_data = Aster
            table_headers = table_data.find_all("th")
            for jeff in table_headers:
                bruh.append(jeff.text)
            tables.append(bruh)
        #Having found the table, putting it all into a dataframe. We use concat to make sure stuff like the Legendary Heroes 
        #decks get all their cards put in the set  
        dataframes = []
        for j in range(0, len(tables)):
            if (('Card number' in tables[j]) and ('Name' in tables[j])) or (('Set number\n' in tables[j]) and ('Name\n' in tables[j])) or (tables[j] == ['Set number\n', 'English name\n', 'Rarity\n', 'Category\n']):
                #a = tables.index(table) #This line is the problem
                table_entries = Destiny[j].find_all('td')
                #Priming the datasets now, depending on if there's a "quantity" column or not
                if (len(tables[j]) == 5):
                    card_number= []
                    name= []
                    rarity= []
                    category = []
                    quantity = []
                    for i in range(0, len(table_entries)):
                        if (i % 5 == 0):
                            card_number.append(table_entries[i].text)
                        elif (i % 5 == 1):
                            name.append(table_entries[i].text)
                        elif (i % 5 == 2):
                            rarity.append(table_entries[i].text)
                        elif (i % 5 == 3):
                            category.append(table_entries[i].text)
                        else:
                            quantity.append(table_entries[i].text)
                    set_content = pd.DataFrame({'Card number': card_number, 'Name': name, 
                                                     'Rarity': rarity ,'Category': category, 'Quantity': quantity})
                    dataframes.append(set_content)
                if (len(tables[j]) == 4):
                    card_number= []
                    name= []
                    rarity= []
                    category = []
                    for i in range(0, len(table_entries)):
                        if (i % 4 == 0):
                            card_number.append(table_entries[i].text)
                        elif (i % 4 == 1):
                            name.append(table_entries[i].text)
                        elif (i % 4 == 2):
                            rarity.append(table_entries[i].text)
                        else:
                            category.append(table_entries[i].text)
                    set_content = pd.DataFrame({'Card number': card_number, 'Name': name, 
                                                     'Rarity': rarity ,'Category': category})
                    dataframes.append(set_content)
                    
        set_content = pd.concat(dataframes, ignore_index = True)
        names = []
        urls = []
        for name in set_content['Name']:
            if (name[0] == '\"') and (name[-1] == '\"'):
                name = name[1:-1]
            name = name.replace(' ', '_')
            url = 'https://yugioh.fandom.com/wiki/' + name
            names.append(name)
            urls.append(url)
        set_content['Name'] = names
        set_content['Card urls'] = urls
        card_infos = []
        for url in list(set_content['Card urls']):
            card_infos.append(card_info(url))
        df_all_cards = pd.DataFrame(card_infos)
        df_xtd = pd.concat([set_content, df_all_cards], axis = 1)
        
        self.set_content = df_xtd
        #Rarity breakdown
        self.rarity_breakdown = dict(self.set_content['Rarity'].value_counts())

In [3]:
DAMA = TCG_set('https://yugioh.fandom.com/wiki/Dawn_of_Majesty')

In [4]:
df = DAMA.set_content

In [5]:
print(df)

    Card number                           Name          Rarity  \
0    DAMA-EN000       "Beetrooper Scout Buggy"     Secret Rare   
1    DAMA-EN001      "Converging Wills Dragon"          Common   
2    DAMA-EN002            "Stardust Synchron"     Secret Rare   
3    DAMA-EN003               "Stardust Trail"          Common   
4    DAMA-EN004               "Despian Comedy"      Super Rare   
..          ...                            ...             ...   
96   DAMA-EN096  "Shinobi Insect Hagakuremino"          Common   
97   DAMA-EN097     "Two Toads with One Sting"          Common   
98   DAMA-EN098           "Trickstar Festival"      Super Rare   
99   DAMA-EN099         "Gouki Finishing Move"          Common   
100  DAMA-EN100              "Stardust Dragon"  Starlight Rare   

                   Category  
0            Effect Monster  
1      Effect Tuner monster  
2      Effect Tuner monster  
3            Effect Monster  
4            Effect Monster  
..                      ...

In [9]:
urls = []
for name in df['Name']:
    name = name[1:-1]
    name = name.replace(' ', '_')
    url = 'https://yugioh.fandom.com/wiki/' + name
    urls.append(url)
df['Card urls'] = urls

In [10]:
print(df)

    Card number                           Name          Rarity  \
0    DAMA-EN000       "Beetrooper Scout Buggy"     Secret Rare   
1    DAMA-EN001      "Converging Wills Dragon"          Common   
2    DAMA-EN002            "Stardust Synchron"     Secret Rare   
3    DAMA-EN003               "Stardust Trail"          Common   
4    DAMA-EN004               "Despian Comedy"      Super Rare   
..          ...                            ...             ...   
96   DAMA-EN096  "Shinobi Insect Hagakuremino"          Common   
97   DAMA-EN097     "Two Toads with One Sting"          Common   
98   DAMA-EN098           "Trickstar Festival"      Super Rare   
99   DAMA-EN099         "Gouki Finishing Move"          Common   
100  DAMA-EN100              "Stardust Dragon"  Starlight Rare   

                   Category                                          Card urls  
0            Effect Monster  https://yugioh.fandom.com/wiki/Beetrooper_Scou...  
1      Effect Tuner monster  https://yugioh.f

In [ ]:
key_list = []
for url in df['Card urls']:
    

In [5]:
print(DAMA.rarity_breakdown)

{'Common': 50, 'Super Rare': 26, 'Ultra Rare': 13, 'Secret Rare': 7, 'Secret RareStarlight Rare': 3, 'Ultra RareStarlight Rare': 1, 'Starlight Rare': 1}


In [6]:
df = DAMA.set_content
df_starlight = df[df['Rarity'].str.contains('Starlight Rare')]

In [19]:
list_minus_rarity = list(df.columns)
print(list_minus_rarity)
list_minus_rarity.remove('Rarity')
print(list_minus_rarity)

['Card number', 'Name', 'Rarity', 'Category']
['Card number', 'Name', 'Category']


['Card number', 'Name', 'Category']


In [18]:
df.loc[df['Rarity'].str.contains('Starlight Rare'), list_minus_rarity].values.flatten().tolist()

['DAMA-EN009',
 '"The Iris Swordsoul"',
 'Effect Monster',
 'DAMA-EN024',
 '"Antihuman Intelligence ME-PSY-YA"',
 'Effect Pendulum Monster',
 'DAMA-EN025',
 '"Protecting Spirit Loagaeth"',
 'Effect Monster',
 'DAMA-EN039',
 '"Shooting Majestic Star Dragon"',
 'Effect Synchro Monster',
 'DAMA-EN100',
 '"Stardust Dragon"',
 'Effect Synchro Monster']

In [8]:
strings = list(df_starlight['Rarity']) 
string_lists =[]
for string in strings:
    for i in range(0, len(string)):
        if (string[i].isupper() == True):
            if (i > 0 and (string[i-1] != ' ')):
                string_list = [string[:i] , string[i:]]
                for string_2 in string_list:
                    
                    
                break
print(string_lists)


[['Secret Rare', 'Starlight Rare'], ['Secret Rare', 'Starlight Rare'], ['Ultra Rare', 'Starlight Rare'], ['Secret Rare', 'Starlight Rare']]


In [9]:
DEM1 = TCG_set('https://yugioh.fandom.com/wiki/Demo_Pack')
DEM2 = TCG_set('https://yugioh.fandom.com/wiki/Demo_Deck_2015')
DEM3 = TCG_set('https://yugioh.fandom.com/wiki/Demo_Deck_2016')

In [10]:
set_info_dict = [DEM1.set_info, DEM2.set_info, DEM3.set_info]

In [11]:
print(set_info_dict)

[{'Medium': 'TCG ', 'Type': 'Demo Deck', 'Prefix(es)': 'DEM1', 'Number of cards': '20', 'Yugioh-Card database\xa0ID': '1131307000 (en)3131302000 (fr)2131302000 (de)4131302000 (it)5131302000 (es)', 'Worldwide Release Date': 'February 12, 2011', 'France, Germany, Italy, and Spain Release Date': 'February 12, 2011'}, {'Medium': 'TCG ', 'Type': 'Demo Deck', 'Prefix(es)': 'DEM2', 'Number of cards': '20', 'Yugioh-Card database\xa0ID': '1131311000 (en)1131311000 (fr)1131311000 (de)1131311000 (it)1131311000 (pt)', 'North America Release Date': 'March 30, 2015', 'France Release Date': 'March 8, 2018', 'Germany Release Date': 'March 8, 2018', 'Italy Release Date': 'March 8, 2018', 'Portugal Release Date': 'June 1, 2015', 'Spain Release Date': 'October 22, 2016'}, {'Medium': 'TCG ', 'Type': 'Demo Deck', 'Prefix(es)': 'DEM3', 'Number of cards': '20', 'Yugioh-Card database\xa0ID': '1131312000 (en)3131307000 (fr)2131307000 (de)4131307000 (it)', 'Europe Release Date': 'January 14, 2016', 'France Rele

In [90]:
string = 'Ultra RareStarlight Rare'
for i in range(0, len(string)):
    if (string[i].isupper() == True):
        if (i > 0 and (string[i-1] != ' ')):
            string = string[:i] + ' ' + string[i:]
            break

In [95]:
string = 'Ultra RareStarlight Rare'
for i in range(0, len(string)):
    if (string[i].isupper() == True):
        if (i > 0 and (string[i-1] != ' ')):
            strings = [string[:i] , string[i:]]
            break

In [97]:
strings

['Ultra Rare', 'Starlight Rare']

In [3]:
#Getting the TCG set links
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
            'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 
            'u', 'v', 'w', 'x', 'y', 'z']
#getting a list of links for each category with the alphabet
links = []
for letter in alphabet:
    links.append('https://yugioh.fandom.com/wiki/Category:North_American_English_sets?from=' + letter)

tcg_card_pages = ['https://yugioh.fandom.com/wiki/Category:North_American_English_sets?from=#']
tcg_card_pages = tcg_card_pages + links
tcg_card_pages.append('https://yugioh.fandom.com/wiki/Category:North_American_English_sets?from=¡')
urls = []
for url in tcg_card_pages:
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    for card_link in soup.find_all(attrs={"class":"category-page__member"}):
        a_tag = card_link.find("a")
        urls.append(a_tag.attrs['href'])

In [ ]:
workable_links = []
for url in urls:
    url = 'https://yugioh.fandom.com/' + url
    workable_links.append(url)
workable_links = list(set(workable_links))
print(len(workable_links))
#Things to remove coz they're irrelevent
for link in workable_links:
    if ('_Value_Box' in link) or ('_Plus' in link) or ('Sealed_Play_Battle_Kit' in link) or ('_Power_Box' in link):
        print(link)
        workable_links.remove(link)
workable_links.remove('https://yugioh.fandom.com//wiki/Legendary_Duelists:_Synchro_Storm') #Not released yet
workable_links.remove('https://yugioh.fandom.com//wiki/2021_Tin_of_Ancient_Battles') #Not released yet
workable_links.remove('https://yugioh.fandom.com//wiki/Structure_Deck_-_Lord_of_the_Storm') #doesn't list set content
workable_links.remove('https://yugioh.fandom.com//wiki/The_Grand_Creators') #Not out yet
workable_links.remove('https://yugioh.fandom.com//wiki/Burst_of_Destiny') #Not out yet
workable_links.remove('https://yugioh.fandom.com//wiki/Structure_Deck:_Cyber_Strike') #Not out yet
workable_links.remove('https://yugioh.fandom.com//wiki/Structure_Deck_-_Invincible_Fortress') #doesn't list set content
workable_links.remove('https://yugioh.fandom.com//wiki/Structure_Deck_-_Spellcaster%27s_Judgment') #doesn't list set content
workable_links.remove('https://yugioh.fandom.com//wiki/Millennium_Pack:_Lite_Edition') #doesn't list set content
workable_links.remove('https://yugioh.fandom.com//wiki/Speed_Duel:_Duel_Academy_Box') #Not out yet
workable_links.remove('https://yugioh.fandom.com//wiki/Hidden_Arsenal:_Chapter_1') #Not out yet
workable_links.remove('https://yugioh.fandom.com//wiki/Duelist_Pack:_Yugi_%26_Kaiba_Special_Edition') #doesn't have any set cards
print(len(workable_links))

In [51]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
            'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 
            'u', 'v', 'w', 'x', 'y', 'z']
#getting a list of links for each category with the alphabet
links = []
for letter in alphabet:
    links.append('https://yugioh.fandom.com/wiki/Category:Worldwide_English_sets?from=' + letter)

tcg_card_pages = ['https://yugioh.fandom.com/wiki/Category:Worldwide_English_sets?from=#']
tcg_card_pages = tcg_card_pages + links
tcg_card_pages.append('https://yugioh.fandom.com/wiki/Category:Worldwide_English_sets?from=¡')
urls = []
for url in tcg_card_pages:
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    for card_link in soup.find_all(attrs={"class":"category-page__member"}):
        a_tag = card_link.find("a")
        urls.append(a_tag.attrs['href'])

233


In [56]:
workable_links_WW = []
for url in urls:
    url = 'https://yugioh.fandom.com/' + url
    workable_links_WW.append(url)
workable_links_WW = list(set(workable_links_WW))
print(len(workable_links_WW))
workable_links_WW.remove('https://yugioh.fandom.com//wiki/Duelist_Pack_Collection_Tin_2009')
workable_links_WW.remove('https://yugioh.fandom.com//wiki/Shonen_Jump_Fifth_Anniversary_Collector%27s_Issue_promotional_card')
print(len(workable_links_WW))

233
231


In [83]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
            'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 
            'u', 'v', 'w', 'x', 'y', 'z']
#getting a list of links for each category with the alphabet
links = []
for letter in alphabet:
    links.append('https://yugioh.fandom.com/wiki/Category:European_English_sets?from=' + letter)

tcg_card_pages = ['https://yugioh.fandom.com/wiki/Category:European_English_sets?from=#']
tcg_card_pages = tcg_card_pages + links
tcg_card_pages.append('https://yugioh.fandom.com/wiki/Category:European_English_sets?from=¡')
urls = []
for url in tcg_card_pages:
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'html.parser')
    for card_link in soup.find_all(attrs={"class":"category-page__member"}):
        a_tag = card_link.find("a")
        urls.append(a_tag.attrs['href'])

In [100]:
workable_links_EU = []
for url in urls:
    url = 'https://yugioh.fandom.com/' + url
    workable_links_EU.append(url)
workable_links_EU = list(set(workable_links_EU))
workable_links_EU.remove('https://yugioh.fandom.com//wiki/HERO_Strike_Elite')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/Structure_Deck:_Cyber_Strike')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/Evolzar_Dolkka_Collectible_Tin')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/The_Grand_Creators')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/Star_Pack_2014_Beginner%27s_Kit')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/Legendary_Duelists:_Synchro_Storm')
workable_links_EU.remove('https://yugioh.fandom.com//wiki/User:Dlamash')
print(len(workable_links_EU))

428


In [4]:
df = pd.read_csv('TCG_set_info_4.csv')

In [13]:
list(df['Wiki link']).remove('https://yugioh.fandom.com//wiki/Structure_Deck_-_Invincible_Fortress')

In [19]:
#TCG_set_info[196] is Duelist_League_Series_2
#TCG_set_info[204] is Duelist Pack Collection Tin
#TCG_set_info[748] is Ultimate Edition 2

In [50]:
print(TCG_set_info[196])
print(TCG_set_info[204])
print(TCG_set_info[745])

{'Medium': 'TCG ', 'Type': 'Duelist League participation cards', 'Prefix(es)': 'DL2', 'Number of cards': '2', 'Yugioh-Card database\xa0ID': '1121202002 (en)', 'Name': 'Duelist League Series 2', 'Wiki link': 'https://yugioh.fandom.com/wiki/Duelist_League_Series_2_participation_cards'}
{'Medium': 'TCG ', 'Type': 'Tin', 'Prefix(es)': 'DPCT', 'Number of cards': '4', 'Cover card': 'Big One WarriorBri Synchron', 'Yugioh-Card database\xa0ID': '15508000 (en)45508000 (it)', 'North America Release Date': 'April 5, 2011', 'Europe Release Date': 'March 31, 2011', 'France, Germany, Italy, and Spain Release Date': 'March 31, 2011', 'Name': 'Duelist Pack Collection Tin 2011', 'Wiki link': 'https://yugioh.fandom.com/wiki/Duelist_Pack_Collection_Tin_2011'}
{'Medium': 'TCG ', 'Type': 'Special Edition', 'Prefix(es)': 'UE01', 'Number of cards': '1', 'Worldwide Release Date': 'November 11, 2006', 'Name': 'Ultimate Edition 2', 'Wiki link': 'https://yugioh.fandom.com/wiki/Ultimate_Edition_2'}


In [51]:
TCG_set_info[204]['Prefix(es)'] = 'DPC5'
TCG_set_info[745]['Prefix(es)'] = 'UE02'

In [37]:
#TCG_set_info = []
#done = []

for link in list(df['Wiki link']):
    print(link)
    card_set = TCG_set(link)
    TCG_set_info.append(card_set.set_info)
    done.append(1)
    print(len(done))

https://yugioh.fandom.com//wiki/Structure_Deck_-_Warrior%27s_Triumph
627
https://yugioh.fandom.com//wiki/Structure_Deck_-_Zombie_Madness
628
https://yugioh.fandom.com//wiki/Structure_Deck:_Cyberse_Link
629
https://yugioh.fandom.com//wiki/Structure_Deck:_Deluxe_Edition
630
https://yugioh.fandom.com//wiki/Structure_Deck:_Dinosaur%27s_Rage
631
https://yugioh.fandom.com//wiki/Structure_Deck:_Freezing_Chains
632
https://yugioh.fandom.com//wiki/Structure_Deck:_Lair_of_Darkness
633
https://yugioh.fandom.com//wiki/Structure_Deck:_Marik_(TCG)
634
https://yugioh.fandom.com//wiki/Structure_Deck:_Mechanized_Madness
635
https://yugioh.fandom.com//wiki/Structure_Deck:_Order_of_the_Spellcasters
636
https://yugioh.fandom.com//wiki/Structure_Deck:_Powercode_Link
637
https://yugioh.fandom.com//wiki/Structure_Deck:_Rokket_Revolt
638
https://yugioh.fandom.com//wiki/Structure_Deck:_Sacred_Beasts
639
https://yugioh.fandom.com//wiki/Structure_Deck:_Seto_Kaiba
640
https://yugioh.fandom.com//wiki/Structure_Dec

752
https://yugioh.fandom.com//wiki/WSJ_Jump_Pack_Spring_2018_promotional_card
753
https://yugioh.fandom.com//wiki/WSJ_Jump_Pack_Spring_2019_promotional_card
754
https://yugioh.fandom.com/wiki/War_of_the_Giants_Reinforcements
755
https://yugioh.fandom.com//wiki/War_of_the_Giants_Reinforcements
756
https://yugioh.fandom.com/wiki/War_of_the_Giants:_Round_2
757
https://yugioh.fandom.com//wiki/War_of_the_Giants:_Round_2
758
https://yugioh.fandom.com//wiki/Warriors%27_Strike_Structure_Deck
759
https://yugioh.fandom.com//wiki/Weekly_Shonen_Jump_Alpha_April_2012_membership_promotional_card
760
https://yugioh.fandom.com//wiki/Weekly_Shonen_Jump_Alpha_December_2012_membership_promotional_card
761
https://yugioh.fandom.com//wiki/Weekly_Shonen_Jump_Alpha_July_2012_membership_promotional_card
762
https://yugioh.fandom.com//wiki/Weekly_Shonen_Jump_Alpha_September_2012_membership_promotional_card
763
https://yugioh.fandom.com//wiki/Weekly_Shonen_Jump_April_2015_membership_promotional_card
764
https:

848
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Day_July_2017_promotional_card
849
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Destiny_Board_Traveler_promotional_cards
850
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Double_Pack_promotional_cards
851
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Duelist_Volume_16_promotional_card
852
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Elemental_Hero_Collection_1
853
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Elemental_Hero_Collection_2
854
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_Forbidden_Memories_promotional_cards
855
https://yugioh.fandom.com/wiki/Yu-Gi-Oh!_GX_Card_Almanac_promotional_cards
856
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_GX_Card_Almanac_promotional_cards
857
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_GX_Duel_Academy_promotional_cards
858
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_GX_Special_Edition
859
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_GX_Spirit_Caller_promotional_cards
860
https://yugioh.fandom.com//wiki/Yu-Gi-Oh!_GX_Tag_Force_prom

In [52]:
df_all_sets = pd.DataFrame(TCG_set_info)

In [53]:
df_2 = pd.concat([df_all_sets, df_structure_decks], ignore_index = True)

In [54]:
df_2.to_csv('TCG_set_info_5.csv')

In [105]:
df = pd.read_csv('TCG_set_info_2.csv')
for col in df.columns:
    if "Unnamed" in col:
        del df[col]
df_2 = pd.concat([df, df_lost_sets], ignore_index = True)
df_2 = df_2.drop_duplicates(ignore_index = True)
df_2.to_csv('TCG_set_info_3.csv')

In [104]:
print(df_2.tail())

    Medium              Type Prefix(es)    Number of cards  \
934    TCG      Booster Pack       TDIL  80 (OCG)100 (TCG)   
935    TCG  Promotional card        PT1                  2   
936    TCG               Tin       MP21                NaN   
937    TCG      Booster Pack       LTGY  80 (OCG)100 (TCG)   
938    TCG      Booster Pack       DAMA                 80   

                                 Cover card  \
934                    Nirvana High Paladin   
935                                     NaN   
936                                     NaN   
937  Number 107: Galaxy-Eyes Tachyon Dragon   
938           Shooting Majestic Star Dragon   

                               Yugioh-Card database ID  \
934  1119000 (ja)11115005 (en)31115005 (fr)21115005...   
935  1121204010 (en)3121203004 (fr)2121204004 (de)4...   
936                                                NaN   
937  1115003 (ja)11112003 (en)31112003 (fr)21112003...   
938  1000008600000 (ja)2000001127000 (en)2000001127...

In [39]:
print(df_2['Name'].tail())

643    Battle Pack 2: War of the Giants
644       Battle Pack 3: Monster League
645          War of the Giants: Round 2
646                          Retro Pack
647                        Retro Pack 2
Name: Name, dtype: object
